## Final Notebook with more features|| CASCADE CUP '22


### 1. Data Cleaning

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=None)

In [3]:
train_labels=data_train["cancelled"]
data_train=data_train.drop(['pickup_time','accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method','cancelled_time',
                            'delivered_time','cancelled','order_id'],axis=1)
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [16]:
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.9)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/100)

###  2. TWEAK THE MODEL

In [17]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(7,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(7, activation='relu'),
    tf.keras.layers.Dense(3, kernel_regularizer='l1_l2'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])
  #opt = tf.keras.optimizers.SGD(lr=0.0001)
  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [18]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=1, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid))

/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-03 14:31:49.218044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


100/100 [==============================] - 1s 8ms/step - loss: 0.4263 - accuracy: 0.9433 - val_loss: 0.2705 - val_accuracy: 0.9879


/Users/aman/miniforge3/lib/python3.9/site-packages/keras/backend.py:4993: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
2022-02-03 14:31:50.234292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [19]:
X_train.shape[0]

270000

In [20]:
model.evaluate(X_test,y_test)

2813/2813 [==============================] - 9s 3ms/step - loss: 0.2707 - accuracy: 0.9881


[0.2706637680530548, 0.9880555868148804]

### 3. CSV SUBMISSION

In [21]:
test=pd.read_csv("data/test.csv")
test_id=test["order_id"]
test=test.drop(['accept_time','allot_time','order_date','order_time','reassignment_reason',
                            'reassigned_order','rider_id','reassignment_method',
                            'cancelled','order_id'],axis=1)
test.to_numpy()
test=np.nan_to_num(test)

In [22]:
ans=model.predict(test)

2022-02-03 14:32:20.236487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [23]:
sum(ans)/len(ans)

array([0.10616458])

In [24]:
max(ans)

array([0.528245], dtype=float32)

In [25]:
data=pd.DataFrame()
ans=np.where(ans > 0.25, 1, 0)
data.insert(0,"order_id",test_id)
data.insert(1,"cancelled",ans)


In [26]:
data.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [27]:
data.to_csv("data/model1_15.csv",index=False)